## Парсер на основе библиотеки Scrapy

In [1]:
import platform
import scrapy
import logging
from scrapy.crawler import CrawlerProcess
import pandas as pd
from w3lib.html import strip_html5_whitespace as shw
from IPython.core.interactiveshell import InteractiveShell
import warnings
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"

### Настройка пайплайнов
Класс `JsonWriterPipeline` создает пайплайн, в который пишет все найденные элементы в формат `JSON`.
Все данные будут сохраняться в два файл: первый с расширением `.json`, второй с расширением `.jl` (в нем каждый элемент json находится на новой странице).

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('data/cian/cian_data.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### Создание класса для парсинга
В списке `start_urls` генерируются URL страниц, из которых нужно извлечь информацию: 
- `name` - название объекта
- `price` - цена объекта

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'file:/Users/alexandrasorokovikova/2 курс/Формальные языки/Проект парсеры/parsing-with-python/tests/cian/test_source/test{0}.html'.format(i) for i in range(1, 50)
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',                               
        'FEED_URI': 'data/cian/cian_data.json'                      
    }
    
    def parse(self, response):
        yield {
            'name': response.xpath("//span[@data-mark='OfferTitle']/span/text()").get(),
            'price': response.xpath("//span[@data-mark='MainPrice']/span/text()").get(),
            }

### Парсинг

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2021-10-19 09:47:48 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-10-19 09:47:48 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-20.2.0-x86_64-i386-64bit
2021-10-19 09:47:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-10-19 09:47:48 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


### Обзор полученных данных

In [5]:
cian_dataset = pd.read_json('data/cian/cian_data.json')
cian_dataset.head(10)

,name,price
0,"1-комн. кв., 41,98 м², 3/10 этаж",14 273 200 ₽
1,"2-комн. кв., 46,9 м², 7/10 этаж",16 884 000 ₽
2,"3-комн. кв., 68,14 м², 4/10 этаж",23 167 600 ₽
3,"4-комн. кв., 124,59 м², 5/10 этаж",33 639 300 ₽
4,"5-комн. кв., 73,4 м², 2/3 этаж",8 200 000 ₽
5,"1-комн. апарт., 38 м², 10/18 этаж",48 000 ₽/мес.
6,Дизайнерская Видовая 2-ка 95м²,85 000 ₽/мес.
7,терраса100 м2.вид.без комиссии,280 000 ₽/мес.
8,Уникальная квартира,135 000 ₽/мес.
9,"Просторная кв, исторический центр",180 000 ₽/мес.


In [6]:
cian_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    47 non-null     object
 1   price   47 non-null     object
dtypes: object(2)
memory usage: 912.0+ bytes


Получили 47 непустых объектов